In [18]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.data import random_split

import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset

device = torch.device('cuda')

In [15]:
class Inception_block(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool):
        super(Inception_block, self).__init__()

        self.branch1 = conv_block(in_channels, out_1x1, kernel_size=1)
        
        self.branch2 = nn.Sequential(
            conv_block(in_channels, red_3x3, kernel_size=1),
            conv_block(red_3x3, out_3x3, kernel_size=3, padding=1)
        )
        
        self.branch3 = nn.Sequential(
            conv_block(in_channels, red_5x5, kernel_size=1),
            conv_block(red_5x5, out_5x5, kernel_size=5, padding=2)
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            conv_block(in_channels, out_1x1pool, kernel_size=1)
        )
    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1)
        

class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(conv_block, self).__init__()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        return self.relu(self.batchnorm(self.conv(x)))
        

In [16]:
class GoogLeNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=4):
        super(GoogLeNet, self).__init__()
        
        self.conv1 = conv_block(in_channels=in_channels, out_channels=64, kernel_size=(7,7), stride =(2,2), padding=(3,3))
        
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding= 1)
        self.conv2 = conv_block(64,192, kernel_size=3, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        #In this order: in)channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool
        self.inception3a = Inception_block(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception_block(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception4a = Inception_block(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception_block(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception_block(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception_block(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception_block(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception5a = Inception_block(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception_block(832, 384, 192, 384, 48, 128, 128)
        
        self.avgpool = nn.AvgPool2d(kernel_size = 7, stride=1)
        self.dropout = nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(1024,4)
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc1(x)
        return x

In [14]:
if __name__ == '__main__':
    x = torch.randn(3, 3, 224, 224)
    model = GoogLeNet()
    print(model(x).shape)

torch.Size([3, 4])


In [19]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # resize image
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = ImageFolder(root="./AugmentedTrain", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [20]:
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        
        for filename in os.listdir(root_dir):
            if filename.endswith('.jpg'):
                img_path = os.path.join(root_dir, filename)
                self.samples.append((img_path, filename))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, filename = self.samples[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, filename


transform = transforms.Compose([
    transforms.Resize((224, 224)),  # resize image
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [21]:
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_subset, val_subset = random_split(train_dataset, [train_size, val_size])


train_loader = DataLoader(train_subset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=4, shuffle=False)

test_dataset = TestDataset(root_dir="./Test/Test/", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [22]:
# 1. Initialize the Model, Loss, and Optimizer
model = GoogLeNet(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of training epochs
num_epochs = 30

for epoch in range(num_epochs):
    correct_train = 0
    total_train = 0
    # 2. Training Loop
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, target)
        
        _, predicted_train = outputs.max(1)
        correct_train += predicted_train.eq(target).sum().item()
        total_train += target.size(0)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    train_accuracy = 100. * correct_train / total_train
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Accuracy: {train_accuracy:.2f}%")
    
    model.eval()
    total_val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss = criterion(outputs, target)
            total_val_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(target).sum().item()

    avg_val_loss = total_val_loss / len(val_loader)
   
    val_accuracy = 100. * correct / len(val_subset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

Epoch [1/30], Step [100/460], Loss: 0.9715
Epoch [1/30], Step [200/460], Loss: 0.9038
Epoch [1/30], Step [300/460], Loss: 1.2358
Epoch [1/30], Step [400/460], Loss: 0.8059
Epoch [1/30], Training Accuracy: 50.54%
Epoch [1/30], Validation Loss: 0.9742, Validation Accuracy: 56.52%
Epoch [2/30], Step [100/460], Loss: 0.8539
Epoch [2/30], Step [200/460], Loss: 0.6499
Epoch [2/30], Step [300/460], Loss: 1.0108
Epoch [2/30], Step [400/460], Loss: 1.1007
Epoch [2/30], Training Accuracy: 53.32%
Epoch [2/30], Validation Loss: 0.9492, Validation Accuracy: 59.57%
Epoch [3/30], Step [100/460], Loss: 1.5389
Epoch [3/30], Step [200/460], Loss: 1.0304
Epoch [3/30], Step [300/460], Loss: 0.9538
Epoch [3/30], Step [400/460], Loss: 0.9812
Epoch [3/30], Training Accuracy: 54.57%
Epoch [3/30], Validation Loss: 0.9595, Validation Accuracy: 54.78%
Epoch [4/30], Step [100/460], Loss: 0.7694
Epoch [4/30], Step [200/460], Loss: 1.1109
Epoch [4/30], Step [300/460], Loss: 0.6923
Epoch [4/30], Step [400/460], Loss

In [23]:
device = torch.device("cuda:0")
model.to(device)
model.eval()  

filenames = []
predictions = []

with torch.no_grad():  
    for data, filename in test_loader:  
        data = data.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)  # Find the class index with the maximum value for each sample
        filenames.extend(filename)
        predictions.extend(predicted.cpu().numpy().tolist())

model.train()  # Set the model back to training mode
import pandas as pd

df_output = pd.DataFrame({
    'Filename': filenames,
    'Prediction': predictions 
})

df_output.to_csv('ggnet_predictions.csv', index=False, header=False) 

In [24]:
class TensorLabelDataset(Dataset):
    def __init__(self, original_dataset):
        self.original_dataset = original_dataset

    def __len__(self):
        return len(self.original_dataset)

    def __getitem__(self, idx):
        data, label = self.original_dataset[idx]
        return data, torch.tensor(label, dtype=torch.long)

In [25]:
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

# Wrap train_subset
tensor_label_train_subset = TensorLabelDataset(train_subset)
train_loader = DataLoader(tensor_label_train_subset, batch_size=4, shuffle=True, drop_last=True) # use tensor_label_train_subset here

val_loader = DataLoader(val_subset, batch_size=4, shuffle=False)

test_dataset = TestDataset(root_dir="./Test/Test/", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [27]:
print(data.shape, target.shape)


torch.Size([1, 3, 224, 224]) torch.Size([1])


In [28]:
class PseudoLabeledDataset(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        data, label = self.data_list[idx]
        return data, torch.tensor(label, dtype=torch.long)  # Explicitly casting to tensor

# Initialize the Model, Loss, and Optimizer
model = GoogLeNet(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of training epochs
num_epochs = 30

# Pseudo-labeling threshold
threshold = 0.95

for epoch in range(num_epochs):
    correct_train = 0
    total_train = 0
    
    # Pseudo-labeling: get the pseudo-labels for the test data
    model.eval()
    pseudo_labels = []
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device)
            outputs = F.softmax(model(data), dim=1)
            _, predicted = outputs.max(1)
            # Only keep predictions with confidence above the threshold
            mask = outputs.max(1)[0] > threshold
            for i in range(len(mask)):
                if mask[i]:
                    pseudo_labels.append((data[i].cpu(), predicted[i].item()))  # Using integer here, but it's casted to tensor when getting item

    # Concatenate pseudo-labeled data with original training data
    pseudo_dataset = PseudoLabeledDataset(pseudo_labels)
    #combined_dataset = ConcatDataset([train_dataset, pseudo_dataset])
    combined_dataset = ConcatDataset([tensor_label_train_subset, pseudo_dataset])

    combined_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)

    # Training Loop with pseudo-labeled data
    model.train()
    for batch_idx, (data, target) in enumerate(combined_loader):
        data, target = data.to(device), target.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(data)
        #loss = criterion(outputs, target.squeeze())  # Use squeeze in case of any singleton dimensions
        loss = criterion(outputs, target)
        
        _, predicted_train = outputs.max(1)
        correct_train += predicted_train.eq(target).sum().item()
        total_train += target.size(0)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(combined_loader)}], Loss: {loss.item():.4f}")

    train_accuracy = 100. * correct_train / total_train
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Accuracy: {train_accuracy:.2f}%")
    
    # Validation Loop
    model.eval()
    total_val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss = criterion(outputs, target)
            total_val_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(target).sum().item()

    avg_val_loss = total_val_loss / len(val_loader)
   
    val_accuracy = 100. * correct / len(val_subset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

Epoch [1/30], Training Accuracy: 52.94%
Epoch [1/30], Validation Loss: 0.9505, Validation Accuracy: 62.17%
Epoch [2/30], Training Accuracy: 56.09%
Epoch [2/30], Validation Loss: 0.9167, Validation Accuracy: 64.13%
Epoch [3/30], Training Accuracy: 58.32%
Epoch [3/30], Validation Loss: 0.9415, Validation Accuracy: 58.04%
Epoch [4/30], Training Accuracy: 58.27%
Epoch [4/30], Validation Loss: 0.9268, Validation Accuracy: 61.09%
Epoch [5/30], Training Accuracy: 59.58%
Epoch [5/30], Validation Loss: 0.9498, Validation Accuracy: 64.57%
Epoch [6/30], Training Accuracy: 59.66%
Epoch [6/30], Validation Loss: 0.8770, Validation Accuracy: 62.83%
Epoch [7/30], Training Accuracy: 60.28%
Epoch [7/30], Validation Loss: 0.8948, Validation Accuracy: 63.70%
Epoch [8/30], Training Accuracy: 58.81%
Epoch [8/30], Validation Loss: 0.9281, Validation Accuracy: 63.70%
Epoch [9/30], Training Accuracy: 59.85%
Epoch [9/30], Validation Loss: 0.9890, Validation Accuracy: 52.83%
Epoch [10/30], Training Accuracy: 59.

In [29]:
device = torch.device("cuda:0")
model.to(device)
model.eval()  

filenames = []
predictions = []

with torch.no_grad():  
    for data, filename in test_loader:  
        data = data.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)  # Find the class index with the maximum value for each sample
        filenames.extend(filename)
        predictions.extend(predicted.cpu().numpy().tolist())

model.train()  # Set the model back to training mode
import pandas as pd

df_output = pd.DataFrame({
    'Filename': filenames,
    'Prediction': predictions 
})

df_output.to_csv('Ggnet_Pl_predictions.csv', index=False, header=False) 
#61%